In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("/kaggle/input/daily-air-quality-dataset-india/air_quality_index.csv")

In [ ]:
### Changing datatypes for date

data["date"] = pd.to_datetime(data["DATE"],dayfirst=True)

### creatng unique dataframe for country

usa_data = data[data["COUNTRY"] == "US"]
india_data = data[data["COUNTRY"] == "IN"]

### creating copy of datasets

india_copy = india_data.copy()
usa_copy = usa_data.copy()

### checking unique cities of india

unique_city_india = list(india_copy["CITY"].unique())

In [ ]:
### checking top 10 maximum pollutant cities of India

def max_pollutant_city(pollutant):
    polu = india_copy[[pollutant,"CITY"]].groupby(["CITY"]).mean().sort_values(by = pollutant,ascending = False).reset_index()
    polu[pollutant] = round(polu[pollutant],2)  
    return polu[:10]

city_polluted = max_pollutant_city("VALUE")      

## plotting bar plot for max polluted cities
plt.figure(figsize = (20,10))
sns.barplot(x = city_polluted["CITY"], y= city_polluted["VALUE"], data = city_polluted, palette="rocket") 
plt.show()                            

By taking the mean of Pollutant value, city Ghāziābād seems to be most polluted city of India.
Let's do some Visualization plots of top 6 cities of India.

In [ ]:
## taking top 6 most polluted cities of India

cities = ["Ghāziābād","Delhi","New Delhi","Lucknow","Muzaffarnagar", "Hāpur"]
sns.set_style('whitegrid')
plt.figure(figsize=(16,18))
i=0
for city in cities:
    p = india_data[india_data["CITY"]==city]
    i +=1
    plt.figure(figsize=(30,10))
    plt.subplot(3,2,i)
    sns.lineplot(x = p["date"], y = p["VALUE"], data = p)
    plt.xlabel(city)
    plt.show()

plt.show()

From this line plot, we can see a sharp spike just before april of 2019, a sharp rise the pollutant value in these cities. 

Particlarly the largest spike is observed in New Delhi and Delhi. 

A similar pattern can be observed in all those cities. A gradual increase in poluution from October to January. 

In [ ]:
### creating lineplot for each cities of India and USA yearly basis

india_copy["year"] = india_copy["date"].dt.year
unique_year = list(india_copy["year"].unique())
unique_year.sort()
sns.set_style("whitegrid")

i=0
for city in cities:
    p = india_copy[india_copy["CITY"]==city]
    i +=1
    plt.figure(figsize = (30,10))
    plt.subplot(3,2,i)
    for year in unique_year:
        p_year = p[p["year"]== year]
        xx = p_year.drop(columns = ["DATE", "date","COUNTRY"])
        xx.reset_index()
        sns.lineplot(x = range(len(xx)), y = xx["VALUE"], data = xx, label = str(p_year["year"].iloc[0]))
        plt.xlabel(city)
        plt.legend(loc = 'best')
    plt.show()
plt.show()

From these yearly plots we can defer that, the pollution pattern is almost similar in each year in those cities. 

In 2019, we saw the biggest spike in all those cities. 

The data is incomplete for 2021, and for 2020, the pollution is also less than 2019 level which can be inferred from the graphs.  

Let's check the distibution plot to check the distribution of pollution in these cities. 

In [ ]:
### creating distplot for each cities of India and USA yearly basis

india_copy["year"] = india_copy["date"].dt.year
unique_year = list(india_copy["year"].unique())
unique_year.sort()
sns.set_style("whitegrid")


i=0
for city in cities:
    p = india_copy[india_copy["CITY"]==city]
    i +=1
    plt.figure(figsize = (30,10))
    plt.subplot(3,2,i)
    j = 0
    for year in unique_year:
        p_year = p[p["year"]== year]
        sns.distplot(p_year["VALUE"], kde = True, hist = False, label = unique_year[j])
        j +=1
        plt.xlabel(city)
        plt.legend()
    plt.show()
plt.show()

From the above distribution plots, we can easily say that in 2020, the distribution plot is speaking about the lockdown effect in those cities as well, where the graph is differerent than the other years. 

Now we will check distribution plot for quarterly basis. The aim is to find out in which quarter the distribution plot is very much differnet than the other months.  


In [ ]:
## creating equal spaced array
quarter_wise = np.arange(0,13,3)

sns.set_style('whitegrid')

for year in unique_year:
    pp = india_copy[india_copy["year"] == year]
    plt.figure()
    i = 0
    for city in cities:
        p = pp[pp["CITY"]==city]
        p["month"] = p["date"].dt.month
    ## segregating data in quaterly basis
        first_quarter = p[(p["month"] >= quarter_wise[0]+1) & (p["month"] <= quarter_wise[1])]
        second_quarter = p[(p["month"] > quarter_wise[1]) & (p["month"] <= quarter_wise[2])]
        third_quarter = p[(p["month"] > quarter_wise[2]) & (p["month"] <= quarter_wise[3])]
        fourth_quarter = p[(p["month"] > quarter_wise[3]) & (p["month"] <= quarter_wise[4])]
        i+=1
        plt.figure(figsize=(30,10))
        plt.subplot(3,2,i)
        sns.distplot(first_quarter["VALUE"], kde = True, hist = False, label = "first_quarter")
        sns.distplot(second_quarter["VALUE"], kde = True, hist = False, label = "second_quarter")
        sns.distplot(third_quarter["VALUE"], kde = True, hist = False, label = "third_quarter")
        sns.distplot(fourth_quarter["VALUE"],kde = True, hist = False,  label = "fourth_quarter")
        plt.xlabel(city)
        plt.legend()
    plt.suptitle("plot of " + str(pp["year"].values[0]) +" " + " dataset")
    plt.show()
    plt.show()
plt.show()
plt.show()

From the quarter plot analysis, we can see that third quarter which comprises of April - June has the lowest pollution in all the cities.

and in the fourth quarter which comprises of October to December, has the highest pollution in all the cities. 

So pollution is increasing in the winter month, we can fairly say that. 

In [ ]:
### Making Predictions on the most polluted state of India - Ghāziābād

Ghāziābād_city = india_copy[india_copy["CITY"] == "Ghāziābād"]

df = Ghāziābād_city.reset_index()["VALUE"]

Ghāziābād_city.drop(columns = ["DATE","COUNTRY","CITY","year"], axis = 1, inplace = True)

training_set = Ghāziābād_city.iloc[:700, 0:1].values
test_set = Ghāziābād_city.iloc[700:, 0:1].values

# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, 700):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print(X_train.shape)

model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 256, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 256, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 256, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 100))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
result = model.fit(X_train, y_train, epochs = 100, batch_size = 16)



In [ ]:
### plotting the loss
plt.figure(figsize = (30,15))
loss = result.history['loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, label='Training loss')
plt.legend()
plt.show()


# Getting the predicted poluution level 
dataset_train = Ghāziābād_city.iloc[:700, 0:1]
dataset_test = Ghāziābād_city.iloc[700:, 0:1]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 249):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

# prediction 
predicted_pollution = model.predict(X_test)
predicted_pollution = sc.inverse_transform(predicted_pollution)

## making a copy of the dataset
dataset_test_copy = dataset_test.copy()
test_data = pd.merge(dataset_test_copy,Ghāziābād_city, left_index = True, right_index = True)

test_data.drop(columns = ["VALUE_x"], inplace = True)

## resting index value for the dataset
dataset_test_copy.reset_index(drop = True, inplace = True)

test_data.reset_index(drop = True, inplace = True)

## plotting actual vs prediction
plt.figure(figsize = (30,10))
plt.plot(test_data.loc[:, 'date'], dataset_test_copy, color = 'black', label = 'Actual Pollution')
plt.plot(test_data.loc[:, 'date'], predicted_pollution, color = 'green', label = 'Predicted Pollution')
plt.title('City Pollution Prediction')
plt.xlabel('Time')
plt.ylabel('Pollution Level Value')
plt.legend()
plt.show()

In [ ]:
n_steps = 7
n_features = 1
x_input = np.array([89,57, 67,85, 162,122,94])
temp_input=list(x_input)
lst_output=[]
i=0
while(i<10):
    
    if(len(temp_input)>7):
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        #print(x_input)
        x_input = x_input.reshape((1, n_steps, n_features))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        yhat = sc.inverse_transform(yhat)
        print("{} day output {}".format(i,yhat))
        temp_input.append(yhat[0][0])
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.append(yhat[0][0])
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        yhat = sc.inverse_transform(yhat)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1
    

print(lst_output)